# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [9]:
# Map star class to numeric rating
RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

def scrape_books(min_rating=4, max_price=20):
    base_url = "http://books.toscrape.com/"
    books = []

    next_page = "catalogue/page-1.html"

    while next_page:
        print(f"Scraping page: {next_page}")
        page_url = base_url + next_page
        res = requests.get(page_url)
        soup = BeautifulSoup(res.text, 'html.parser')

        for article in soup.select('article.product_pod'):
            title = article.h3.a['title']

            # ⭐ Safe price extraction
            price_text = article.select_one('.price_color').text
            match = re.search(r'[\d.]+', price_text)
            if not match:
                print(f"[WARNING] Couldn't extract price from: {price_text}")
                continue
            price = float(match.group())

            # Rating extraction
            rating_class = article.p.get('class')[1]  # e.g. 'Four'
            rating = RATING_MAP.get(rating_class, 0)

            # Filter early
            if rating < min_rating or price > max_price:
                continue

            # Link to book detail page
            detail_relative_url = article.h3.a['href'].replace('../../../', '')
            detail_url = base_url + 'catalogue/' + detail_relative_url

            detail_res = requests.get(detail_url)
            detail_soup = BeautifulSoup(detail_res.text, 'html.parser')

            # Extract additional data
            upc = detail_soup.select_one('th:contains("UPC") + td')
            upc = upc.text if upc else "N/A"

            description_tag = detail_soup.select_one('#product_description ~ p')
            description = description_tag.text.strip() if description_tag else ""

            availability_tag = detail_soup.select_one('.availability')
            availability = availability_tag.text.strip() if availability_tag else "Unknown"

            genre_tag = detail_soup.select('ul.breadcrumb li a')
            genre = genre_tag[-1].text.strip() if len(genre_tag) >= 3 else "Unknown"

            books.append({
                "UPC": upc,
                "Title": title,
                "Price (£)": price,
                "Rating": rating,
                "Genre": genre,
                "Availability": availability,
                "Description": description
            })

            # Be nice to the site
            time.sleep(0.5)

        # Move to the next page
        next_button = soup.select_one('li.next > a')
        next_page = "catalogue/" + next_button['href'] if next_button else None

    # Create and return DataFrame
    return pd.DataFrame(books)

In [10]:
df = scrape_books(min_rating=4, max_price=20)
print(df.head())

Scraping page: catalogue/page-1.html


/opt/anaconda3/lib/python3.12/site-packages/soupsieve/css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


Scraping page: catalogue/page-2.html
Scraping page: catalogue/page-3.html
Scraping page: catalogue/page-4.html
Scraping page: catalogue/page-5.html
Scraping page: catalogue/page-6.html
Scraping page: catalogue/page-7.html
Scraping page: catalogue/page-8.html
Scraping page: catalogue/page-9.html
Scraping page: catalogue/page-10.html
Scraping page: catalogue/page-11.html
Scraping page: catalogue/page-12.html
Scraping page: catalogue/page-13.html
Scraping page: catalogue/page-14.html
Scraping page: catalogue/page-15.html
Scraping page: catalogue/page-16.html
Scraping page: catalogue/page-17.html
Scraping page: catalogue/page-18.html
Scraping page: catalogue/page-19.html
Scraping page: catalogue/page-20.html
Scraping page: catalogue/page-21.html
Scraping page: catalogue/page-22.html
Scraping page: catalogue/page-23.html
Scraping page: catalogue/page-24.html
Scraping page: catalogue/page-25.html
Scraping page: catalogue/page-26.html
Scraping page: catalogue/page-27.html
Scraping page: catal